### 多辆车的路径规划 VRP：
#### 条件：经过中国33个城市，一共4辆车，每辆车最大行驶10000公里
#### 目标：使得每辆车的行驶里程数更接近

In [1]:
import pandas as pd 

In [2]:
from ortools.constraint_solver import pywrapcp

In [3]:
from ortools.constraint_solver import routing_enums_pb2

In [8]:
class vrp(object):
    #唯一的参数为汽车数量，默认是1
    def __init__(self, num_vehicles = 1):
        #设置城市名
        self.df = pd.read_excel('cities.xlsx')
        self.num_vehicles = num_vehicles
        
    
    # 设置数据
    def create_data_model(self):
        data = {}
        temp = pd.read_excel('distance.xlsx',index_col=0)
        
        #将读入的距离除以1000就是两个城市间的公里数，存入字典中
        data['distance_matrix'] = temp.values / 1000
        #print(data['distance_matrix'])
        data['num_vehicles'] = self.num_vehicles #车的数量
        data['depot'] = 0 #下标
        return data
    
    def get_solution(self,manager, routing, solution):
        #print('总行驶里程: {} 公里'.format(solution.ObjectiveValue()))
        
        #记录每辆车的里程
        distance_list = []
        #记录每辆车的路径规划
        route_list = []
        
        #遍历每辆车
        for vehicle_id in range(self.num_vehicles):
            #初始化距离为0
            route_distance = 0
            #针对第vehicle_id个车进行规划
            route = []
            
            # Start后面是 vehicle_id
            #从起始节点出发
            index = routing.Start(vehicle_id)

            # 判断这辆车的路径是否结束
            while not routing.IsEnd(index):
                # 使用indextonode将manager中的index转换为distance_matrix中的index
                index_show = manager.IndexToNode(index)
                #添加到route
                route.append(index_show)
                previous_index = index

                #走到下一个节点
                index = solution.Value(routing.NextVar(index))

                #针对vehicle = 0，统计从previous_index到index节点的距离
                route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
            route_list.append(route)
            distance_list.append(route_distance)
            
        return route_list, distance_list
    
    #添加距离约束
    def add_distance_dimension(self, routing, transit_callback_index):
        # 添加距离约束
        dimension_name = 'Distance'
        routing.AddDimension(
            transit_callback_index,
            0,  # no slack
            10000,  # 车辆最大行驶距离
            True,  # start cumul to zero
            dimension_name)
        distance_dimension = routing.GetDimensionOrDie(dimension_name)
        
        # 尽量减少车辆之间的最大距离, spancost的系数越大，则几辆车的行驶距离越接近
        #但是超过一百之后，150，200得到的结果和100接近，甚至略有不如
        distance_dimension.SetGlobalSpanCostCoefficient(100)

    
    def work(self):
        # 初始化数据
        data = self.create_data_model()

        # 创建路线管理，tsp_size（城市数量）, num_vehicles（车的数量）, depot（原点）
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                               data['num_vehicles'], data['depot'])

        # 创建 Routing Model.
        routing = pywrapcp.RoutingModel(manager)

        # 注册距离，计算两点之间的距离，输入manager中的两个节点index
        def distance_callback(from_index, to_index):
            # Convert from routing variable Index to distance matrix NodeIndex.
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]
        
        #注册距离函数
        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        # Define cost of each arc.
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
        #添加距离维度
        self.add_distance_dimension(routing, transit_callback_index)
        
        # 设置参数策略，Setting first solution heuristic.
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        #定义最优化问题的目标
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        # 求解路径规划
        solution = routing.SolveWithParameters(search_parameters)
        # 输出结果
        route_list, distance_list = self.get_solution(manager, routing, solution)
        
        return route_list, distance_list
    


#### 运行一下模块求解问题

In [9]:
if __name__ == '__main__':
    model = vrp(num_vehicles = 4) #本次一共有4辆车
    route_list, distance_list = model.work()

#### 显示具体的路线

In [11]:
route_list

[[0, 7, 18, 19, 17, 22, 6],
 [0, 21],
 [0, 5, 20, 23, 24, 26, 27, 25, 12, 32, 11, 31, 30, 13, 16],
 [0, 14, 28, 29, 10, 9, 8, 15, 1, 4, 2, 3]]

##### 显示每辆车行驶的距离，大概在6800上下

In [12]:
distance_list

[6341, 7096, 6749, 6845]